In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleSyntheticDataGenerator(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, features):
        super(SimpleSyntheticDataGenerator, self).__init__()
        self.sequence_length = sequence_length
        self.features = features
        
        # Define the network structure
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, sequence_length * features)
    
    def forward(self, x):
        # Simple forward pass with activation
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # Reshape the output to [batch_size, sequence_length, features]
        x = x.view(-1, self.sequence_length, self.features)
        return x

def generate_data_with_nn(batch_size, input_size, hidden_size, sequence_length, features, device='cpu'):
    # Initialize the data generator model
    generator = SimpleSyntheticDataGenerator(input_size, hidden_size, sequence_length, features).to(device)
    
    # Generate random noise as input
    random_noise = torch.randn(batch_size, input_size, device=device)
    
    # Generate synthetic time series data through the model
    synthetic_data = generator(random_noise)
    
    # For simplicity, labels are generated randomly here; adjust as needed
    labels = torch.randint(0, 2, (batch_size,), dtype=torch.long, device=device)
    
    return synthetic_data, labels

# Parameters for synthetic data generation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
input_size = 100  # Dimensionality of the input random noise
hidden_size = 128  # Capacity of the generator model
sequence_length = 30  # Length of the generated time series
features = 2  # Number of features in the time series

# Generate synthetic data and labels
X_synthetic, y_synthetic = generate_data_with_nn(batch_size, input_size, hidden_size, sequence_length, features, device=device)
print(f"Generated Synthetic Data Shape: {X_synthetic.shape}")  # Expected: [64, 30, 2]
print(f"Generated Synthetic Labels Shape: {y_synthetic.shape}")  # Expected: [64]
X_synthetic = X_synthetic.detach()  # This ensures X_synthetic is a leaf tensor
y_synthetic = y_synthetic.detach()  
X_synthetic.requires_grad = False
y_synthetic.requires_grad = False
# Assume X_synthetic and y_synthetic are generated
# Ensures y_synthetic is a leaf tensor

# Proceed with training


Generated Synthetic Data Shape: torch.Size([64, 30, 2])
Generated Synthetic Labels Shape: torch.Size([64])


In [30]:
import numpy as np
import torch
from cnn_transformer import CNNTransformer


def generate_time_series_data(n_samples=1000, dimensions=2, sequence_length=30):
    # Generate random time series data
    # Shape: [n_samples, sequence_length, dimensions]
    X = np.random.randn(n_samples, sequence_length, dimensions)
    
    # For simplicity in generating labels, use the sum of values across all dimensions and time steps
    # This is just one way to generate labels based on time series data; adjust according to your needs
    labels = np.sum(X, axis=(1, 2)) > 0  # Label based on the sum being positive or negative
    labels = labels.astype(int)
    
    # Convert to PyTorch tensors
    X_tensor = torch.tensor(X, dtype=torch.float32)
    labels_tensor = torch.tensor(labels, dtype=torch.long)
    
    return X_tensor, labels_tensor

X, y = generate_time_series_data()


In [31]:
X_synthetic

tensor([[[-0.5265, -0.0449],
         [ 0.4183,  0.2150],
         [ 0.2953,  0.3668],
         ...,
         [ 0.0210,  0.0845],
         [-0.1138, -0.0956],
         [ 0.4182, -0.2301]],

        [[ 0.1569,  0.2218],
         [ 0.1902, -0.5205],
         [-0.0538,  0.3558],
         ...,
         [-0.0647, -0.0881],
         [ 0.2100,  0.2904],
         [-0.2477, -0.1842]],

        [[-0.1542,  0.3030],
         [ 0.0830,  0.1237],
         [ 0.3992, -0.2318],
         ...,
         [-0.1460, -0.1430],
         [-0.1822, -0.1553],
         [-0.0917,  0.0371]],

        ...,

        [[-0.0129,  0.3371],
         [ 0.1563, -0.0699],
         [ 0.2558,  0.4522],
         ...,
         [-0.0668,  0.2322],
         [ 0.1484, -0.4111],
         [ 0.2976,  0.2927]],

        [[-0.3457,  0.2712],
         [ 0.6759, -0.6507],
         [ 0.3519,  0.5558],
         ...,
         [-0.0461, -0.0824],
         [-0.1298, -0.5942],
         [-0.5149,  0.3210]],

        [[-0.5667,  0.2400],
       

In [32]:
from torch.utils.data import TensorDataset, DataLoader

# Assuming X and y are your generated data and labels
dataset = TensorDataset(X_synthetic, y_synthetic)
batch_size = 16  # You can adjust this
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


In [35]:
model = CNNTransformer(
    random_seed=0, 
    lookback=1,  # Not directly applicable due to the structure of linear data, but set for consistency
    device="cuda:0" if torch.cuda.is_available() else "cpu",  # Adjust according to your setup
    normalization_conv=True, 
    filter_numbers=[2, 16],  # First number adjusted to match input features
    attention_heads=4, 
    use_convolution=True,
    hidden_units_factor=8, 
    dropout=0.50, 
    filter_size=2, 
    use_transformer=True,
    num_classes=2
)


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed


In [36]:
num_epochs = 1000  # Adjust this

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:

        inputs, labels = inputs.to(model.device), labels.to(model.device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')


Epoch 1/1000, Loss: 0.7282
Epoch 2/1000, Loss: 0.7478
Epoch 3/1000, Loss: 0.6746
Epoch 4/1000, Loss: 0.6142
Epoch 5/1000, Loss: 0.6852
Epoch 6/1000, Loss: 0.6154
Epoch 7/1000, Loss: 0.6537
Epoch 8/1000, Loss: 0.6419
Epoch 9/1000, Loss: 0.6516
Epoch 10/1000, Loss: 0.6332
Epoch 11/1000, Loss: 0.6210
Epoch 12/1000, Loss: 0.6114
Epoch 13/1000, Loss: 0.6420
Epoch 14/1000, Loss: 0.6569
Epoch 15/1000, Loss: 0.5662
Epoch 16/1000, Loss: 0.5672
Epoch 17/1000, Loss: 0.5624
Epoch 18/1000, Loss: 0.5893
Epoch 19/1000, Loss: 0.5821
Epoch 20/1000, Loss: 0.5759
Epoch 21/1000, Loss: 0.5793
Epoch 22/1000, Loss: 0.5395
Epoch 23/1000, Loss: 0.5547
Epoch 24/1000, Loss: 0.5475
Epoch 25/1000, Loss: 0.5724
Epoch 26/1000, Loss: 0.5078
Epoch 27/1000, Loss: 0.5293
Epoch 28/1000, Loss: 0.5298
Epoch 29/1000, Loss: 0.5745
Epoch 30/1000, Loss: 0.5107
Epoch 31/1000, Loss: 0.5017
Epoch 32/1000, Loss: 0.5265
Epoch 33/1000, Loss: 0.4724
Epoch 34/1000, Loss: 0.5043
Epoch 35/1000, Loss: 0.4478
Epoch 36/1000, Loss: 0.4377
E

KeyboardInterrupt: 